In [ ]:
# pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 7.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm

from collections import Counter

In [3]:
def delete_punctuation_marks_tag(word):
    word = word.replace(",","")
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace("!","")
    word = word.replace(":","")
    word = word.replace("?","")
    word = word.replace("~","")
    word = word.replace("\\","")
    word = word.replace("\"","")
    word = word.replace(";","")
    return word.lower()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
eat_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restaurant_info.csv')
eat_review = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restaurant_review.csv')


In [5]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,김북순큰남비집,서울특별시 강남구 압구정로2길 15 신사아이빌 1F,한정식 / 백반 / 정통 한식,5000,25,4.0,37.520699,127.019975,88,"맛있음. #돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : ..."
1,두부공작소,서울특별시 강남구 강남대로160길 21,한정식 / 백반 / 정통 한식,5000,16,3.8,37.519766,127.020477,166,점점 망해가는 느낌잠점 책상도 끈적끈적해지는것 같고...맛은 그럭저럭인편. 가격에 ...
2,빌라드스파이시,서울특별시 강남구 도산대로13길 14-3,기타 한식,5000,38,3.6,37.519027,127.023549,322,떡볶이가 너무 좋아서 사먹어봤어요. 즉석 떡볶이가 참 맛있어요ㅎ약간 매운 맛인데도 ...
3,스쿨푸드,서울특별시 강남구 도산대로 139,기타 한식,5000,11,3.5,37.518213,127.023711,398,오랜만에 스쿨푸드에서 배달시켜 먹었어요. 15년 전 즈음엔 가로수길에 와서 스쿨푸드...
4,송옥,서울특별시 강남구 강남대로154길 43 2F,국수 / 면 요리,5000,11,3.1,37.518625,127.022385,376,판메밀(2판) / 돈가스. 한국식 판메밀을 판매하는 곳으로 역사도 있는 집으로 보인...
...,...,...,...,...,...,...,...,...,...,...
768,요술포차,서울특별시 강남구 테헤란로19길 5,일반 주점,25000,5,NaN,37.500368,127.033440,528,예약하러고 전화 3번 했을 때 알아봤어야 했는데... 저녁에 3테이블정도밖에 없을 ...
769,전통주갤러리,서울특별시 강남구 테헤란로5길 51-20,전통 주점 / 포차,25000,2,NaN,37.501505,127.027861,591,다양한 전통주를 시음할 수 있는 공간전통주 무료시음회도 하고 판매도 하는 곳입니다....
770,더블린테라스,서울특별시 강남구 강남대로94길 27-15 태리빌딩,치킨 / 호프 / 펍,25000,1,NaN,37.500657,127.029090,148,강남역에 위치한 맥주펍. CGV 뒷골목 더블린 말고 주택가 쪽에 올라가면 위치해있는...
771,술술술,서울특별시 강남구 강남대로98길 12-5,일반 주점,25000,2,NaN,37.500376,127.027961,381,오랫동안 강남역 바닥에서 술을 마셨지만 못들어본곳이었던 이곳. 왠만한 안주는 다 좋...


In [6]:
eat_review

,res_name,user_name,review,rating,p_id,u_id
0,모터시티,Mike Kim,Love motor city pizza,5,243,559
1,김북순큰남비집,철준,맛있음,5,88,3735
2,김북순큰남비집,돼후니,"#돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : 3.7점 ...",5,88,1961
3,김북순큰남비집,호리,[신사/김북순 큰남비집] 5/5오 좋다! 소리가 절로 나온 찌개 맛집이오~개인적으로...,5,88,4100
4,김북순큰남비집,하몽이,누구나 아는 맛 그러나 모두가 즐기는 맛 근처에서 뜨끈한 국물에 흰 쌀밥 생각나면 ...,5,88,3990
...,...,...,...,...,...,...
17130,깐부치킨,Crisp.,깐부 처음가서 그런데 구운마늘치킨 먹음.. 맛은 있대 좀 많이 기름진거 같았어요.,3,95,160
17131,깐부치킨,Miriam Park,We ordered the garlic soy chicken. The chicken...,3,95,570
17132,모터시티,호두꽈자,포치즈홀스맨 24900그릴드 치킨 아보카도 샐러드 16900 (발사믹)[공간]생긴 ...,3,243,4095
17133,모터시티,이영훈,디트로이트 스타일 피자 처음먹어보는데 맛있어요 ㅋㅋ 가격은 좀 나가지만 토마토 소스...,5,243,3341


In [7]:
eat_review_by_user = pd.DataFrame(index = eat_review['u_id'].unique(), columns=['reviews'])

for id in tqdm(eat_review['u_id'].unique()):
    review_all = ''
    for review in eat_review.loc[eat_review['u_id']==id,'review']:
        if type(review) != float:
            review_all += review+'. '
    eat_review_by_user.loc[id,'reviews'] = review_all

  0%|          | 0/4181 [00:00<?, ?it/s]

In [8]:
eat_info['tags']=''
eat_review_by_user['tags']=''

In [9]:
eat_info.head()

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews,tags
0,김북순큰남비집,서울특별시 강남구 압구정로2길 15 신사아이빌 1F,한정식 / 백반 / 정통 한식,5000,25,4.0,37.520699,127.019975,88,"맛있음. #돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : ...",
1,두부공작소,서울특별시 강남구 강남대로160길 21,한정식 / 백반 / 정통 한식,5000,16,3.8,37.519766,127.020477,166,점점 망해가는 느낌잠점 책상도 끈적끈적해지는것 같고...맛은 그럭저럭인편. 가격에 ...,
2,빌라드스파이시,서울특별시 강남구 도산대로13길 14-3,기타 한식,5000,38,3.6,37.519027,127.023549,322,떡볶이가 너무 좋아서 사먹어봤어요. 즉석 떡볶이가 참 맛있어요ㅎ약간 매운 맛인데도 ...,
3,스쿨푸드,서울특별시 강남구 도산대로 139,기타 한식,5000,11,3.5,37.518213,127.023711,398,오랜만에 스쿨푸드에서 배달시켜 먹었어요. 15년 전 즈음엔 가로수길에 와서 스쿨푸드...,
4,송옥,서울특별시 강남구 강남대로154길 43 2F,국수 / 면 요리,5000,11,3.1,37.518625,127.022385,376,판메밀(2판) / 돈가스. 한국식 판메밀을 판매하는 곳으로 역사도 있는 집으로 보인...,


In [10]:
eat_review_by_user.head()

,reviews,tags
559,Love motor city pizza. Finally got to try the ...,
3735,맛있음.,
1961,"#돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : 3.7점 ...",
4100,[신사/김북순 큰남비집] 5/5오 좋다! 소리가 절로 나온 찌개 맛집이오~개인적으로...,
3990,누구나 아는 맛 그러나 모두가 즐기는 맛 근처에서 뜨끈한 국물에 흰 쌀밥 생각나면 ...,


In [11]:
# with open('../data/stop_words.txt','r',-1,'utf-8') as f:
#     stop_words = set(f.read().split('/'))

In [12]:

stopwords = ['맛','더','것','곳','가격','때','밥','진짜','정도','건대','집','여기','음식','좀','처음','수',
             '생각','사람','방문','주문','정말','친구','바로','다른','가게','바','이','조금','또',
             '내','로우','거','하나','메뉴','요리','그','점','가지','별로','식당','한번','추천','삼선','보고','그냥','꽤','보통','나','입','제','직원',
             '다시','말','왜','그릇','무게','꼭','다음','개','느낌','아주','자리','거리','때문','리뷰',
             '중','도','웨이','팅','일단','사진','저','계속','가장','제일','걸','듯','안','건',
             '명','간이','이유','역시','편이','추가','편','재','먹기','거의','시간','기도','테이블','분','마',
             '담','날','근처','손','땐','못','완전','애정','항상','첫','제대로','입맛','이번','먹음',
             '궁합','최고','원래','가면','예전','뭔가','평점','만','한국','다만','기대','배','막','오위','시작',
             '종류','후','냄새','중간','해먹','번','뒤','배달',
             '그램','점점','갑자기','감','살짝','물','콤','색',
             '간','사실','강추','최애','자체','이전','카','골목','음식점','상태','금방','위생','용서','조리','질','요','그때',
             '널','리지','약간','앞','시로','마루','무난','기본','취향','깨','양','줄','수도','개인','매장','식사','두',
             '생','산다','사을','자극','다소','둘','전체','끼','서비스','이이','맛집','아부','얼','비주','오픈','저녁','으;',
             '마끼','대기','운','재료','퀄리티','점심','소스','포장','내부','할머니','토마','식감','동네','일','혼자','요즘','어마','가득','해',
             '저희','볼','인기','뭐','말씀','평일','세','도착','사이즈','일반','밖','간장','후회','인분','손님','달','자주','함',
             '굳이','모든','매우','거기','인생','학교','크기','분위기','절대','주변','무조건','인정','푹','식','옆','주차','위치','존맛','스타일',
             '이모','무엇','밤','새벽','길','기준','알','자꾸','전형','무척','차','베이스','불어','임','위','음','이제','얘기','이해','비','기분',
             '지금','덜','답','북적','신장','개업','끝','방송','위해','법','특유','모습','비교','부분','웨이팅','옥','은','동이','팀','베','부',
             '하의','의사','케이','쿠라','인테리어','마당','대비','인','워낙','쿠니','반','데','마음','개조','미리','단점','하이','','','','',
             '양념','가성','구의역','술집','양도','세트','직접','시징','가족','체인점','안나',
             '이집','저집','적','시','물이','모두','갓','글','수가','집의','합리','부집','준','해도','갈이'
             ]

In [13]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 270 kB/s 
     |████████████████████████████████| 453 kB 67.7 MB/s 


In [14]:
import konlpy
Okt = konlpy.tag.Okt()

def get_tags_to_txt(text):
    Okt_morphs = Okt.pos(str(text)) 
    

    Noun_words = []
    for word, pos in Okt_morphs:
        if pos == 'Noun':
            Noun_words.append(word)
    
    unique_Noun_words = set(Noun_words)
    for word in unique_Noun_words:
        if word in stopwords:  #stopwords
            while word in Noun_words: Noun_words.remove(word)  # 최종결과 : Noun_word
    
    if type(text) != str:
        return ''
    tags = Noun_words
    
    
    tags = ','.join(tags)
    
    return tags

In [15]:
for i in tqdm(eat_info.index):
    eat_info.loc[i,'tags'] = get_tags_to_txt(eat_info.loc[i,'reviews'])
    print(get_tags_to_txt(eat_info.loc[i,'reviews']))
    
for i in tqdm(eat_review_by_user.index):
    eat_review_by_user.loc[i,'tags'] = get_tags_to_txt(eat_review_by_user.loc[i,'reviews'])
    print(get_tags_to_txt(eat_review_by_user.loc[i,'reviews']))

Output hidden; open in https://colab.research.google.com to view.

In [16]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews,tags
0,김북순큰남비집,서울특별시 강남구 압구정로2길 15 신사아이빌 1F,한정식 / 백반 / 정통 한식,5000,25,4.0,37.520699,127.019975,88,"맛있음. #돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : ...","한식,벽,냄비,뚜껑,연예,싸인,주말,점심시간,평,김치,들깨,순두부찌개,들깨,순두부찌..."
1,두부공작소,서울특별시 강남구 강남대로160길 21,한정식 / 백반 / 정통 한식,5000,16,3.8,37.519766,127.020477,166,점점 망해가는 느낌잠점 책상도 끈적끈적해지는것 같고...맛은 그럭저럭인편. 가격에 ...,"잠점,책상,인편,부대,햄,순두부,순두부,볶음,정식,두부김치,고기,정식,순두부,콩국수..."
2,빌라드스파이시,서울특별시 강남구 도산대로13길 14-3,기타 한식,5000,38,3.6,37.519027,127.023549,322,떡볶이가 너무 좋아서 사먹어봤어요. 즉석 떡볶이가 참 맛있어요ㅎ약간 매운 맛인데도 ...,"떡볶이,즉석,떡볶이,튀김,튀김,옷,내용물,떡뽂,만족,청담,대요,가로수길,청담,야채,..."
3,스쿨푸드,서울특별시 강남구 도산대로 139,기타 한식,5000,11,3.5,37.518213,127.023711,398,오랜만에 스쿨푸드에서 배달시켜 먹었어요. 15년 전 즈음엔 가로수길에 와서 스쿨푸드...,"스쿨,푸드,전,즈음,가로수길,스쿨,푸드,기억,오징어,날치,스팸,구성,스페셜,마리,돼..."
4,송옥,서울특별시 강남구 강남대로154길 43 2F,국수 / 면 요리,5000,11,3.1,37.518625,127.022385,376,판메밀(2판) / 돈가스. 한국식 판메밀을 판매하는 곳으로 역사도 있는 집으로 보인...,"판,메밀,판,돈가스,판,메밀,판매,역사,판,메밀,판이,장점,메밀,툼,메밀,육수,소바..."
...,...,...,...,...,...,...,...,...,...,...,...
768,요술포차,서울특별시 강남구 테헤란로19길 5,일반 주점,25000,5,NaN,37.500368,127.033440,528,예약하러고 전화 3번 했을 때 알아봤어야 했는데... 저녁에 3테이블정도밖에 없을 ...,"예약,전화,벨,체류,셀프,별문제,지경,요약,이자카야,펍,어딘가,안주,안주,안고,역,..."
769,전통주갤러리,서울특별시 강남구 테헤란로5길 51-20,전통 주점 / 포차,25000,2,NaN,37.501505,127.027861,591,다양한 전통주를 시음할 수 있는 공간전통주 무료시음회도 하고 판매도 하는 곳입니다....,"전통주,공간,전통주,무료,회도,판매,전통주,관심,책,소개,전통주,종,무료,음회,신청..."
770,더블린테라스,서울특별시 강남구 강남대로94길 27-15 태리빌딩,치킨 / 호프 / 펍,25000,1,NaN,37.500657,127.029090,148,강남역에 위치한 맥주펍. CGV 뒷골목 더블린 말고 주택가 쪽에 올라가면 위치해있는...,"강남역,맥주,펍,뒷골목,더블린,주택가,쪽,더블린,테라스,맥주,유럽,펍,테라스,좌석,썩"
771,술술술,서울특별시 강남구 강남대로98길 12-5,일반 주점,25000,2,NaN,37.500376,127.027961,381,오랫동안 강남역 바닥에서 술을 마셨지만 못들어본곳이었던 이곳. 왠만한 안주는 다 좋...,"강남역,바닥,술,안주,강남,안주,착,한편,소주,쟁반,냉면,과일,보쌈,매콤,냉면,과일..."


In [17]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restuarant_info.csv')

In [18]:
# eat_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/eat_info.csv')

In [19]:
eat_review_by_user

,reviews,tags
559,Love motor city pizza. Finally got to try the ...,"초밥,연어,개미,국밥,삼겹살,갈비,삼겹살,삼겹살,목살,삼겹살,죽,새우,죽"
3735,맛있음.,
1961,"#돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : 3.7점 ...","한식,벽,냄비,뚜껑,연예,싸인,주말,점심시간,평,김치,들깨,순두부찌개,들깨,순두부찌..."
4100,[신사/김북순 큰남비집] 5/5오 좋다! 소리가 절로 나온 찌개 맛집이오~개인적으로...,"신사,김북순,남비,오,소리,절로,찌개,이오,맵,김치찌개,감칠맛,선호,맵,김치,목,살..."
3990,누구나 아는 맛 그러나 모두가 즐기는 맛 근처에서 뜨끈한 국물에 흰 쌀밥 생각나면 ...,"누구,국물,쌀밥,누구,냄비,찌개,류,여타,통일,안해,장점,스트레스,매운갈비찜,콩나물..."
...,...,...
1899,망고 밍글링 2019. 모터시티 잠실점. 좋은 사람들. 좋은 음식들. 피맥회식하기 ...,"망고,밍글링,모터,시티,잠실,피맥,회식"
3576,사내에 깐부치킨 빠가있어서 회식할때마다 2차로 여기가요.맛있어요. 근처 치킨집들중에...,"사내,치킨,빠,회식,차로,치킨"
494,오늘저녁운 치킨치킨!야근을 위해 오늘은.. 치킨으로 충전! 깐부는 크리스피가 진리지...,"오늘,치킨,치킨,야근,오늘,치킨,충전,크리스,피,진리,저그"
2949,여기 깐부 맛있어요 생맥도 추천.,생맥


In [20]:
eat_review_by_user.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restaurant_review_by_user.csv')

In [21]:
# pip install gensim

In [24]:
# w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
import gensim
w2v = gensim.models.Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/ko/ko.bin')

In [25]:
All_tags = set()
for tags in eat_info['tags']:
    All_tags.update(set(tags.split(',')))

In [26]:
new_df = pd.DataFrame()
for word in All_tags:
    try:
        f_vec = w2v.wv.get_vector(word)
        new_df[word] = f_vec
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """


In [27]:
new_df

,우등,하노이,고체,해독,탈취,리터,배설,반사,파인애플,낭,...,봉투,부의,인터넷,퀄,선택,지구,고서,유심,셋,파트
0,-0.133101,0.582772,2.048321,0.763388,-0.700502,0.002703,0.658763,0.622436,0.664829,2.153435,...,0.821774,-0.007761,-1.299362,0.173373,-0.823235,-0.383554,-0.333862,0.253627,0.089487,-0.341430
1,0.015218,-0.055364,0.456558,-0.145077,-0.082380,-0.281732,0.100559,1.319129,-0.417164,0.788175,...,0.758512,1.182292,-0.691568,-0.284560,0.690061,0.482325,0.150791,-0.050086,-1.963602,0.493731
2,0.089959,-0.744545,-0.526120,-1.811379,-2.122451,0.756639,-0.239261,-1.399728,0.034407,-0.448426,...,-0.169761,-0.240301,-1.018206,0.247866,0.656642,-2.477071,1.167450,0.379501,0.870633,-0.773752
3,-0.157071,0.142963,0.082103,-0.133257,-1.683666,0.117667,-0.152940,0.258155,0.474218,-0.557678,...,-0.519178,-0.626846,2.201069,1.408319,-0.430824,1.311376,0.722472,0.007509,0.186222,1.642203
4,0.139028,0.479873,-0.524326,0.077752,1.211982,-1.003797,0.158986,-0.147460,0.434755,0.831402,...,-0.355400,1.496389,0.451871,-0.015304,0.242138,-2.046342,0.544437,-0.168832,0.315651,-0.133786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.395637,0.731413,-0.172949,-0.580262,-1.171840,-0.692338,0.285347,0.272083,0.173712,0.934520,...,0.046185,1.550989,-0.589118,-0.832004,0.686047,1.209031,2.084871,0.050111,-1.185570,-1.899653
196,0.331789,-0.734488,-0.216373,0.962274,0.588215,0.306150,0.656952,1.903594,-0.017053,0.334412,...,-0.427686,1.465063,0.200503,-0.299223,0.294399,-0.677982,-0.383616,0.174178,-0.155134,-1.602148
197,0.502069,0.307871,0.786491,0.731281,-0.554321,0.106365,0.203431,2.083698,0.456606,0.279256,...,-0.729385,-0.250721,-0.215050,-0.154694,1.601527,1.361745,-0.698662,-0.248149,-1.006969,0.734402
198,-0.238399,-1.142094,0.650518,0.981943,0.696793,-0.460539,1.096980,0.218980,0.212619,0.229836,...,0.142350,0.032681,1.850324,0.433363,-0.253688,-2.292791,0.273672,0.491679,-1.145872,1.553783


In [28]:
x = new_df.T
cluster_count=120
model = KMeans(n_clusters=cluster_count)
model.fit(x)
model.predict(x)
Y=x.copy()
Y['kmeans_id'] = model.predict(x)

In [29]:
km = list(Y['kmeans_id'])
max_count = 0
max_count_id = ''
for i in range (0, cluster_count):
    if km.count(i) > max_count:
        max_count = km.count(i);
        max_count_id = i;
    print(i,': ', km.count(i))
max_count_id

0 :  1
1 :  382
2 :  45
3 :  37
4 :  36
5 :  54
6 :  26
7 :  101
8 :  132
9 :  294
10 :  15
11 :  57
12 :  47
13 :  32
14 :  83
15 :  14
16 :  64
17 :  25
18 :  108
19 :  5
20 :  5
21 :  32
22 :  85
23 :  20
24 :  64
25 :  44
26 :  48
27 :  25
28 :  77
29 :  37
30 :  32
31 :  42
32 :  32
33 :  3
34 :  248
35 :  22
36 :  68
37 :  29
38 :  28
39 :  38
40 :  89
41 :  436
42 :  2
43 :  20
44 :  3
45 :  2
46 :  58
47 :  39
48 :  5
49 :  39
50 :  402
51 :  4
52 :  124
53 :  1
54 :  4
55 :  1
56 :  37
57 :  30
58 :  42
59 :  1
60 :  44
61 :  110
62 :  68
63 :  69
64 :  38
65 :  51
66 :  59
67 :  18
68 :  12
69 :  81
70 :  49
71 :  77
72 :  37
73 :  1291
74 :  57
75 :  2
76 :  66
77 :  39
78 :  30
79 :  32
80 :  18
81 :  29
82 :  67
83 :  26
84 :  33
85 :  43
86 :  13
87 :  54
88 :  17
89 :  36
90 :  2
91 :  67
92 :  46
93 :  49
94 :  45
95 :  1
96 :  122
97 :  39
98 :  307
99 :  1
100 :  173
101 :  2
102 :  38
103 :  72
104 :  15
105 :  31
106 :  94
107 :  57
108 :  22
109 :  51
110 :  89
111

73

In [30]:
km_df = pd.DataFrame()
km_df['group_'+str(max_count_id)] = pd.Series(list(Y[Y['kmeans_id'] == max_count_id].index))
for i in range(0, cluster_count):
    col_name = 'group_'+str(i)
    words = list(Y[Y['kmeans_id'] == i].index)
    km_df[col_name] = pd.Series(words)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [31]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
km_df.fillna(0)

Output hidden; open in https://colab.research.google.com to view.

In [32]:
# remove groups we don't need
# rm_groups = [17, 26, 50, 63]
# for i in rm_groups:
#     group_name = 'group_'+str(i);
#     stop_words.update(set(km_df[group_name].dropna()))
# with open('../data/stop_words.txt','w',-1,'utf-8') as f:
#     f.write('/'.join(stop_words))

In [33]:
del eat_info['Unnamed: 0']
eat_info.head()

KeyError: ignored

In [34]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restuarant_info_tags.csv')

In [35]:
res_matrix = pd.DataFrame(index=eat_info.index,columns=list(km_df.columns))
user_matrix = pd.DataFrame(index=eat_review_by_user.index,columns=list(km_df.columns))
res_matrix = res_matrix.fillna(0)
user_matrix = user_matrix.fillna(0)

In [36]:
res_matrix.head()

,group_73,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
user_matrix.head()

,group_73,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
559,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1961,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3990,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
for id in tqdm(res_matrix.index):
    tags = Counter(eat_info.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                res_matrix.loc[id,'group_'+str(num)] += tags[tag]

  0%|          | 0/773 [00:00<?, ?it/s]

In [39]:
res_matrix.head()

,group_73,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
0,3,0,8,0,1,1,0,0,5,15,2,0,0,1,0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,3,0,0,0,1,1,0,1,2,1,0,1,0,0,1,0,0,0,25,0,4,0,0,0,0,1,4,0,1,4,1,0,0,1,8,0,0,0,0,0,0,1,0,1,1,0,3,0,0,1,0,0,10,0,0,0,2,0,1,1,4,0,0,7,1,4,0,2,0,0,1,0,0,1,1,0,1,3,1,4,0,0,0,0,1,0,0
1,10,0,10,3,1,0,1,0,4,45,2,0,0,2,0,0,1,0,0,3,0,0,0,2,0,2,0,0,1,0,1,1,0,2,0,8,0,4,2,0,0,3,1,0,0,0,0,0,1,0,0,51,0,3,0,0,0,1,1,0,0,1,2,1,2,0,3,3,1,0,1,0,0,1,0,0,4,1,0,2,0,3,0,0,0,17,0,1,0,4,0,0,3,0,1,0,2,2,3,0,9,0,1,1,0,0,3,0,2,1,0,1,4,1,1,0,1,4,0,0
2,65,0,28,6,0,5,4,3,3,113,8,2,19,5,4,1,4,11,1,21,0,0,1,4,0,5,6,4,0,2,3,2,3,4,0,8,1,15,2,6,1,17,22,0,1,1,0,4,2,0,2,300,1,22,0,0,0,3,6,12,0,1,21,5,2,1,5,13,2,0,2,0,4,0,1,1,6,4,5,8,0,2,1,0,13,96,0,1,1,21,0,4,17,0,1,0,25,6,34,0,34,0,17,9,0,1,28,5,3,5,6,3,2,3,12,4,2,43,0,3
3,31,0,14,0,0,5,1,0,2,21,4,8,1,2,0,0,2,0,0,19,0,0,2,1,1,1,0,1,1,1,2,30,0,0,0,4,1,2,0,7,1,9,22,0,0,2,0,11,0,1,0,57,0,8,0,0,0,1,1,2,0,1,8,1,7,1,6,43,6,1,4,1,1,1,3,1,3,1,2,5,0,2,2,0,4,36,0,1,0,7,1,1,4,0,4,0,5,1,5,0,32,0,4,4,1,1,5,1,2,3,7,2,6,3,4,3,2,15,1,0
4,32,0,15,1,2,3,2,0,4,25,9,0,8,6,1,0,0,3,0,12,0,0,0,1,0,3,0,1,0,1,0,0,0,3,0,2,0,6,1,1,0,11,1,0,1,0,0,4,0,0,0,70,0,15,0,0,0,0,2,5,0,0,9,1,19,0,3,3,1,1,2,0,1,4,2,3,5,1,1,3,0,0,1,0,2,10,0,4,0,2,1,0,7,3,5,0,5,3,9,0,42,0,6,3,2,2,3,0,2,5,11,3,10,1,1,0,4,11,0,0


In [40]:
for id in tqdm(user_matrix.index):
    tags = Counter(eat_review_by_user.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                user_matrix.loc[id,'group_'+str(num)] += tags[tag]


  0%|          | 0/4181 [00:00<?, ?it/s]

In [ ]:
user_matrix.head()

In [ ]:
user_matrix.reset_index()

In [43]:
user_matrix.rename(columns ={'index':'u_id'})

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
user_matrix.head()

In [47]:
user_matrix.shape

(4181, 120)

In [48]:
user_matrix.set_index('u_id', drop=True, inplace=True)


KeyError: ignored

In [49]:
km_df.count()

group_73     1291
group_0         1
group_1       382
group_2        45
group_3        37
group_4        36
group_5        54
group_6        26
group_7       101
group_8       132
group_9       294
group_10       15
group_11       57
group_12       47
group_13       32
group_14       83
group_15       14
group_16       64
group_17       25
group_18      108
group_19        5
group_20        5
group_21       32
group_22       85
group_23       20
group_24       64
group_25       44
group_26       48
group_27       25
group_28       77
group_29       37
group_30       32
group_31       42
group_32       32
group_33        3
group_34      248
group_35       22
group_36       68
group_37       29
group_38       28
group_39       38
group_40       89
group_41      436
group_42        2
group_43       20
group_44        3
group_45        2
group_46       58
group_47       39
group_48        5
group_49       39
group_50      402
group_51        4
group_52      124
group_53        1
group_54  

In [50]:
eat_review_score = pd.DataFrame(eat_review, columns=['u_id','p_id','rating'])
eat_review_score.sort_values(by=['u_id'], axis=0,inplace=True)
eat_review_score.reset_index(drop = True, inplace=True)
eat_review_score.head()

,u_id,p_id,rating
0,0,491,5
1,0,689,5
2,0,491,0
3,0,306,3
4,0,633,0


In [51]:
res_matrix.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restuarant_res_matrix.csv')
user_matrix.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restuarant_user_matrix.csv')
eat_review_score.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restuarant_score_board.csv',index=False)
km_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restuarant_eat_cluster.csv',index=False)
km_df.count().to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restuarant_group_count.csv')